Torus Geodesic Benchmark
========================
Compares three ODE solvers (RK4, RK45, DOP853) on periodic geodesics.

Strategy for error measurement
--------------------------------
For a periodic geodesic the curve closes on itself after one period T*.
We find T* numerically (for the reference orbit) and then compare
Y(T*) vs Y(0) for each solver / step-size combination.

  periodicity_error = ||Y(T*) - Y(0)||

This error accumulates with every step, so a coarser h gives a larger error.


In [1]:
import time
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use("TkAgg")   # change to "Qt5Agg" / "Agg" / "MacOSX" as needed
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp, quad
from scipy.optimize import brentq


In [2]:
R = 2.0   # major radius
r = 0.7   # minor radius


def geodesic(t, Y):
    u, up, v, vp = Y
    A = R + r * np.cos(u)
    return [
        up,
        -(A * np.sin(u) / r) * vp**2,
        vp,
        +2 * (r * np.sin(u) / A) * up * vp,   # + sign (Clairaut-conserving)
    ]


In [3]:
def _half_integrals(k):
    """(Tu_half, Tv_half) for one u half-oscillation."""
    u_turn = np.arccos(np.clip((k - R) / r, -1 + 1e-9, 1 - 1e-9))
    def f_u(u):
        A = R + r * np.cos(u)
        return r / np.sqrt(max(1 - k**2 / A**2, 1e-30))
    def f_v(u):
        A = R + r * np.cos(u)
        return k * r / (A**2 * np.sqrt(max(1 - k**2 / A**2, 1e-30)))
    Tu_h, _ = quad(f_u, -u_turn, u_turn, limit=500, points=[0.0])
    Tv_h, _ = quad(f_v, -u_turn, u_turn, limit=500, points=[0.0])
    return Tu_h, Tv_h

def _winding(k):
    Tu_h, Tv_h = _half_integrals(k)
    return 2 * Tv_h / (2 * np.pi)

def find_periodic_geodesic(p, q, tol=1e-11):
    """
    Return (k, T_full, Y0) for the (p:q) periodic geodesic, or None if
    the winding ratio is outside the range accessible for this torus.
    """
    target = p / q
    k_lo, k_hi = R - r + 1e-4, R + r - 1e-4
    if (_winding(k_lo) - target) * (_winding(k_hi) - target) > 0:
        return None   # not accessible
    k = brentq(lambda kk: _winding(kk) - target, k_lo, k_hi, xtol=tol)
    Tu_h, _ = _half_integrals(k)
    T_full = q * 2 * Tu_h
    u_turn = np.arccos(np.clip((k - R) / r, -1, 1))
    vp0 = k / (R + r * np.cos(u_turn))**2
    Y0 = [u_turn, 0.0, 0.0, vp0]
    return k, T_full, Y0

# Human-readable labels for common winding numbers
GEODESIC_LABELS = {
    (1, 1): "1:1 — diagonal",
    (2, 3): "2:3 — gentle spiral",
    (3, 4): "3:4 — medium spiral",
    (4, 5): "4:5 — dense spiral",
    (3, 5): "3:5 — wide winding",
    (4, 3): "4:3 — fast v-wind",
    (4, 7): "4:7 — long period",
    (5, 7): "5:7 — complex",
}


In [4]:

def rk4_solve(Y0, t_end, h):
    """Fixed-step classical RK4."""
    n = max(1, int(round(t_end / h)))
    h_act = t_end / n
    t_arr = np.linspace(0.0, t_end, n + 1)
    y = np.zeros((n + 1, 4))
    y[0] = Y0
    for i in range(n):
        ti, yi = t_arr[i], y[i]
        k1 = h_act * np.array(geodesic(ti,             yi))
        k2 = h_act * np.array(geodesic(ti + h_act/2,   yi + k1/2))
        k3 = h_act * np.array(geodesic(ti + h_act/2,   yi + k2/2))
        k4 = h_act * np.array(geodesic(ti + h_act,     yi + k3))
        y[i + 1] = yi + (k1 + 2*k2 + 2*k3 + k4) / 6
    return t_arr, y

def scipy_solve(Y0, t_end, max_step, method):
    """Adaptive scipy solver (RK45 or DOP853)."""
    sol = solve_ivp(geodesic, [0, t_end], Y0, method=method,
                    max_step=max_step, rtol=1e-9, atol=1e-12)
    return sol.t, sol.y.T


In [5]:
SOLVERS = {
    "RK4 (custom)":  lambda Y0, t_end, h: rk4_solve(Y0, t_end, h),
    "RK45 (scipy)":  lambda Y0, t_end, h: scipy_solve(Y0, t_end, h, "RK45"),
    "DOP853 (scipy)":lambda Y0, t_end, h: scipy_solve(Y0, t_end, h, "DOP853"),
}

H_VALUES = [1.0, 0.5, 0.2, 0.1, 0.05, 0.02, 0.01, 0.005, 0.001]

def run_benchmark(Y0, T_period, h_values=H_VALUES, label=""):
    Y0_arr = np.array(Y0)
    k_true = (R + r * np.cos(Y0_arr[0]))**2 * Y0_arr[3]

    records = []
    print(f"\n{'─'*90}")
    print(f"  Geodesic: {label}   T* = {T_period:.4f}")
    print(f"{'─'*90}")
    print(f"  {'Solver':<20} {'h':>8} {'Steps':>8} {'Time(s)':>9} "
          f"{'Period err':>13} {'Speed err':>12} {'Clairaut err':>14}")
    print(f"  {'─'*18} {'─'*8} {'─'*8} {'─'*9} {'─'*13} {'─'*12} {'─'*14}")

    for solver_name, solver_fn in SOLVERS.items():
        for h in h_values:
            t0_wall = time.perf_counter()
            t_arr, y_arr = solver_fn(list(Y0), T_period, h)
            elapsed = time.perf_counter() - t0_wall

            u_arr  = y_arr[:, 0]
            up_arr = y_arr[:, 1]
            vp_arr = y_arr[:, 3]
            A_arr  = R + r * np.cos(u_arr)

            # Periodicity error
            delta = y_arr[-1] - Y0_arr
            delta[2] = (delta[2] + np.pi) % (2 * np.pi) - np.pi   # wrap v
            period_err = np.linalg.norm(delta)

            # Speed conservation
            speed = np.sqrt(r**2 * up_arr**2 + A_arr**2 * vp_arr**2)
            speed_err = np.max(np.abs(speed - 1.0))

            # Clairaut conservation
            clairaut_err = np.max(np.abs(A_arr**2 * vp_arr - k_true))

            n_steps = len(t_arr) - 1
            records.append({
                "Solver":          solver_name,
                "h / max_step":    h,
                "Steps":           n_steps,
                "Time (s)":        round(elapsed, 5),
                "Periodicity err": period_err,
                "Speed err":       speed_err,
                "Clairaut err":    clairaut_err,
            })
            print(f"  {solver_name:<20} {h:>8.4f} {n_steps:>8} {elapsed:>9.4f} "
                  f"{period_err:>13.3e} {speed_err:>12.3e} {clairaut_err:>14.3e}")

    return pd.DataFrame(records)

In [6]:
def _torus_surface():
    U = np.linspace(0, 2*np.pi, 200)
    V = np.linspace(0, 2*np.pi, 200)
    U, V = np.meshgrid(U, V)
    return ((R + r*np.cos(U))*np.cos(V),
            (R + r*np.cos(U))*np.sin(V),
            r*np.sin(U))

def _to_xyz(u, v):
    return ((R + r*np.cos(u))*np.cos(v),
            (R + r*np.cos(u))*np.sin(v),
            r*np.sin(u))

def plot_geodesics(cases, n_cols=2):
    """One 3-D panel per geodesic."""
    n = len(cases)
    n_rows = (n + n_cols - 1) // n_cols
    fig = plt.figure(figsize=(7*n_cols, 6*n_rows))
    Xg, Yg, Zg = _torus_surface()

    for idx, (label, _, Y0, T_full) in enumerate(cases):
        sol = solve_ivp(geodesic, [0, T_full], Y0, method="DOP853",
                        rtol=1e-12, atol=1e-14, max_step=0.01)
        x, y, z = _to_xyz(sol.y[0], sol.y[2])

        ax = fig.add_subplot(n_rows, n_cols, idx+1, projection="3d")
        ax.plot_surface(Xg, Yg, Zg, alpha=0.20, cmap="viridis", edgecolor="none")
        ax.plot(x, y, z, linewidth=1.8, color="crimson")
        ax.set_title(f"{label}\nT* = {T_full:.2f}", fontsize=9)
        ax.set_xlabel("X"); ax.set_ylabel("Y"); ax.set_zlabel("Z")
        ax.set_xlim(-(R+r), R+r); ax.set_ylim(-(R+r), R+r); ax.set_zlim(-r, r)
        ax.set_box_aspect([2*(R+r), 2*(R+r), 2*r])

    fig.suptitle("Periodic geodesics on the torus  (one full period each)",
                 fontsize=12)
    plt.tight_layout()
    return fig

def plot_convergence(df, title=""):
    """Log-log plots for all three error metrics."""
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    metrics = ["Periodicity err", "Speed err", "Clairaut err"]
    ylabels = ["‖Y(T*) − Y₀‖", "max |speed − 1|", "max |k(t) − k₀|"]
    colors  = {"RK4 (custom)": "#2E86AB",
               "RK45 (scipy)": "#E84855",
               "DOP853 (scipy)": "#3BB273"}
    markers = {"RK4 (custom)": "o",
               "RK45 (scipy)": "s",
               "DOP853 (scipy)": "^"}

    hs = np.array(sorted(df["h / max_step"].unique()))

    for ax, metric, ylabel in zip(axes, metrics, ylabels):
        for solver, grp in df.groupby("Solver"):
            grp = grp.sort_values("h / max_step")
            ax.loglog(grp["h / max_step"], grp[metric],
                      marker=markers[solver], color=colors[solver],
                      label=solver, linewidth=2, markersize=6)

        # Reference slopes anchored to RK4 data
        rk4 = df[df["Solver"]=="RK4 (custom)"].sort_values("h / max_step")
        if len(rk4):
            ref_h  = rk4["h / max_step"].values[-3]
            ref_v  = rk4[metric].values[-3]
            ax.loglog(hs, ref_v*(hs/ref_h)**4, "k--", alpha=0.4, lw=1.2, label="O(h⁴)")
            ax.loglog(hs, ref_v*(hs/ref_h)**8, "k:",  alpha=0.4, lw=1.2, label="O(h⁸)")

        ax.set_xlabel("h  (step-size / max_step)")
        ax.set_ylabel(ylabel)
        ax.set_title(metric)
        ax.legend(fontsize=8)
        ax.grid(True, which="both", ls="--", alpha=0.35)

    fig.suptitle(f"Convergence — {title}", fontsize=11)
    plt.tight_layout()
    return fig

def plot_efficiency(df, title=""):
    """Wall-clock time vs periodicity error."""
    fig, ax = plt.subplots(figsize=(9, 5))
    colors = {"RK4 (custom)": "#2E86AB",
              "RK45 (scipy)": "#E84855",
              "DOP853 (scipy)": "#3BB273"}
    for solver, grp in df.groupby("Solver"):
        ax.scatter(grp["Time (s)"], grp["Periodicity err"],
                   label=solver, color=colors[solver], s=60, zorder=3)
        for _, row in grp.iterrows():
            ax.annotate(f"h={row['h / max_step']}", fontsize=6.5, alpha=0.7,
                        xy=(row["Time (s)"], row["Periodicity err"]),
                        xytext=(4, 4), textcoords="offset points")
    ax.set_xscale("log"); ax.set_yscale("log")
    ax.set_xlabel("Wall time (s)")
    ax.set_ylabel("Periodicity error ‖Y(T*) − Y₀‖")
    ax.set_title(f"Efficiency (time vs accuracy) — {title}")
    ax.legend()
    ax.grid(True, which="both", ls="--", alpha=0.35)
    plt.tight_layout()
    return fig

In [7]:
# ── Choose which (p, q) winding numbers to benchmark ──────────────
# Higher p+q = more complex geodesic = harder for integrators.
# Accessible range for R=2, r=0.7: winding ratio ≈ [0.51, 1.44]
#
# All accessible options (T < 100):
#   (1,1)  diagonal        T ≈  13.8
#   (2,3)  gentle spiral   T ≈  32.0
#   (3,4)  medium spiral   T ≈  46.1
#   (3,5)  wide winding    T ≈  49.6
#   (4,3)  fast v-wind     T ≈  49.8
#   (4,5)  dense spiral    T ≈  60.0
#   (4,7)  long period     T ≈  66.9
# ──────────────────────────────────────────────────────────────────
BENCHMARK_CASES_PQ = [
    (1, 1),   # tribial case
    (2, 3),   # moderate difficulty
    (3, 4),   # harder
    (4, 7),   # long period, most demanding
]

# ── Build case list ────────────────────────────────────────────────
cases = []
for pq in BENCHMARK_CASES_PQ:
    result = find_periodic_geodesic(*pq)
    if result is None:
        print(f"  Warning: {pq} winding not accessible for R={R}, r={r}")
        continue
    k, T_full, Y0 = result
    label = GEODESIC_LABELS.get(pq, f"{pq[0]}:{pq[1]} winding")
    cases.append((label, pq, Y0, T_full))
    print(f"Found {label}:  k={k:.6f}  T*={T_full:.4f}")

# ── Reference closure check ────────────────────────────────────────
print("\n── Reference closure (DOP853, tol=1e-12) ──")
for label, _, Y0, T_full in cases:
    sol = solve_ivp(geodesic, [0, T_full], Y0, method="DOP853", rtol=1e-12, atol=1e-14, max_step=0.005)
    err = np.linalg.norm(sol.y[:, -1] - np.array(Y0))
    print(f"  {label}: {err:.2e}")

# ── Geodesic plot ──────────────────────────────────────────────────
fig_geo = plot_geodesics(cases)

# ── Benchmarks ────────────────────────────────────────────────────
all_dfs = []
figs_conv = []
figs_eff  = []
for label, pq, Y0, T_full in cases:
    df = run_benchmark(Y0, T_full, H_VALUES, label=label)
    df["Geodesic"] = label
    df["T_period"] = round(T_full, 4)
    all_dfs.append(df)
    figs_conv.append(plot_convergence(df, title=label))
    figs_eff.append(plot_efficiency(df,   title=label))

# ── Save CSV ───────────────────────────────────────────────────────
combined = pd.concat(all_dfs, ignore_index=True)
csv_path = "torus_geodesic_benchmark.csv"
combined.to_csv(csv_path, index=False)
print(f"\nAll results saved → {csv_path}")
print(combined.to_string(index=False))

plt.show()

Found 1:1 — diagonal:  k=1.371618  T*=13.7957
Found 2:3 — gentle spiral:  k=1.711335  T*=32.0452
Found 3:4 — medium spiral:  k=1.552426  T*=46.1280
Found 4:7 — long period:  k=2.097049  T*=66.9158

── Reference closure (DOP853, tol=1e-12) ──
  1:1 — diagonal: 6.28e+00
  2:3 — gentle spiral: 1.26e+01
  3:4 — medium spiral: 1.88e+01
  4:7 — long period: 2.51e+01

──────────────────────────────────────────────────────────────────────────────────────────
  Geodesic: 1:1 — diagonal   T* = 13.7957
──────────────────────────────────────────────────────────────────────────────────────────
  Solver                      h    Steps   Time(s)    Period err    Speed err   Clairaut err
  ────────────────── ──────── ──────── ───────── ───────────── ──────────── ──────────────
  RK4 (custom)           1.0000       14    0.0006     3.937e-02    3.735e-03      1.065e-02
  RK4 (custom)           0.5000       28    0.0011     1.447e-03    2.093e-04      2.958e-04
  RK4 (custom)           0.2000       69  